Progress Report 1
----

**Team Members**

Yaqian Cheng, Department of Statistical Science

Mengrun Li, Department of Statistical Science

**Github repository**

<https://github.com/cici7941/Sta_663_Statistical_Computation_Final_Project>

**Choice of paper** 

*Scalable K-Means++*

**Abstract**

*K-means* is one of the most popular clustering methods. A good initialization of *k-means* is essential for obtaining the global optimal solution and efficiency. However, there are two main obstacles with traditional *k-means* method. One is theoretical inefficiency and the other one is that its final solution is locally optimal. A better algorithm, *k-means++* addresses the second problem with an improved initialization procedure of the cluster centers. But this *k-means++* initialization is not parallelizable, because the selection for the *i*th center depends on the previous *i-1* centers [1]. Therefore, *k-means||*, a parallelizable version of *k-means++*, has been raised, which can both improve the final solution and run faster. In this report, we implemented the algorithm in the paper "Scalable K-Means++" in Python, compared the clustering cost and runtime between *k-means*, *k-means++* and *k-means||*, performed tests for main functions, profiled the performance of the algorithm and identified bottlenecks, and performed optimization using Cython. We then apply *k-means||* to a massive dataset to evaluate its performance.

**Outline**

1. Introduction
2. Algorithm  
    2.1 K-Means  
    2.2 K-Means++  
    2.3 K-Means||  
3. Code Testing
4. Profiling and Optimization
5. Application and Comparison

In [4]:
import scipy.linalg as la
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from numba import jit, njit
import operator as op
from functools import reduce

In [5]:
# helper functions
def euc_dist(x, y):
    return la.norm(x-y)

def d(x, Y):
    dist = [euc_dist(x, yi) for yi in Y]
    return min(dist)

def cost(Y, C):
    return sum(d(yi, C)**2 for yi in Y)

**Algorithm 1** *k-means++(k)* initialization 
1. $C$ $\leftarrow$ sample a point uniformly at random from $X$  
2. **while** |$C$| < $k$ **do**  
3. $\,\,$ Sample $x$ $\in$ $X$ with probability $\frac{d^2(x,C}{\psi_X(C)}$  
4. $\,\,$ $C$ $\leftarrow$ $C$ $\cup$ $\{x\}$  
5. **end while**  

In [6]:
# K-Means++
def kmeans_plus(X, k):
    idx = np.random.choice(X.shape[0], 1)
    C = X[idx, :]
    while(C.shape[0] < k):
        cost_X = cost(X, C)
        prob = [d(xi,C)**2/cost_X for xi in X]
        Ct = X[np.random.choice(X.shape[0], size=1, p=prob),:]
        if(Ct.tolist() not in C.tolist()):
            C = np.r_[C, Ct]
    return C

**Initialization algorithm: k-means||**  
*k-means||* uses an oversampling factor *l* = $\Omega$(k), which is unlike *k-means++*. Intuitively, *l* should be thought of as $\Theta$(k). This initialization algorithm picks an initial center uniformly from the dataset and computes $\psi$, here is initial cost of the clustering of this selection. Then do log$\psi$ iterations and in each iteration, it samples each x with probability *l*$d^2(x,C)/\psi_X(C)$ given current set C of centers. If the point is sampled, it will be added to C and the quantity $\phi_X(C)$ updated and interation continued. The expected number of points chosen in each iteration is l. Since there is oversampling factor, thus the number of points in C will be more than k. To reduce the number of centers, weight each point in C by the number of points in X whose distance to this point is shorter than it of any other point in C. And then recluster the weighted points to obtain k centers.

**Algorithm 2** *k-means||(k,l)* initialization  
1. $C$ $\leftarrow$ sample a point uniformly at random from $X$  
2. $\psi$ $\leftarrow$ $\phi_X(C)$  
3. **for** O(log$\psi$) times **do**  
      $\,\,$ $C'$ $\leftarrow$ sample each point $x \in X$ independently with probability $p_x = \frac{l*d^2(x,C)}{\phi_X(C)}$  
      $\,\,$ $C$ $\leftarrow$ $C$ $\cup$ $C'$  
   **end for**  
4. For x $\in$ $C$, set $w_x$ to be the number of points in X closer to x than any other point in C  
5. Recluster the weighted points in C into k clusters  

In [7]:
#helper function for scalable k-means++ sampling
def scalable_kmeans_sample(X, C, cost_X, l):
    Ct = np.empty([0,C.shape[1]])
    prob = list(map(lambda x:l*d(x,C)**2/cost_X, X))
    for j in range(len(X)):
            if(prob[j] > 1):
                point = 1
            else:
                point = np.random.binomial(1,prob[j],size = 1)
            if point == 1 and (X[j,:].tolist() not in C.tolist()):
                Ct = np.r_[Ct, [X[j,:]]]
    return np.r_[C, Ct]

In [8]:
#helper function for scalable k-means++ weighting
def weight(X,C):
    w = []
    for c in C:
        distx_c = np.array([euc_dist(x,c) for x in X])
        distx_otherc = np.array([d(x,C) for x in X])
        count = len(np.where(distx_c==distx_otherc)[0])
        w.append(count)
    return w

In [9]:
###K-means|| speed up
def scalable_kmeans_plus(X, k, l):
    ##Sample a point uniformly at random from X
    idx = np.random.choice(X.shape[0],1)
    C = X[idx,:]
    cost_0 = cost(X, C)
    for i in range(int(round(np.log(cost_0)))):
        cost_X = cost(X,C)
        if(cost_X == 0):
            break
        C = scalable_kmeans_sample(X, C, cost_X, l)
    w = weight(X,C)
    w_prob = w/np.sum(w)
    centroid = np.random.choice(range(C.shape[0]),k,p = w_prob,replace = False)
    return C[centroid]

### Optimization

In [12]:
%load_ext cython

In [13]:
%%cython -a
cimport cython
cimport numpy as np
from libc.math cimport sqrt, pow
from numpy.math cimport INFINITY

@cython.boundscheck(False)
@cython.wraparound(False)
def euc_dist_cython(double[:]x, double[:]y):
    cdef int i, n
    cdef double dist = 0
    n = x.shape[0]
    for i in range(n):
        dist += pow(x[i] - y[i], 2)
    return sqrt(dist)

def d_cython(double[:]x, double[:,:]Y):
    cdef int i, n
    cdef double dist
    cdef double min_dist = INFINITY
    n = Y.shape[0]
    for i in range(n):
        dist = euc_dist_cython(x, Y[i,:])
        if dist < min_dist:
            min_dist = dist
    return min_dist

def cost_cython(double[:,:]Y, double[:,:]C):
    cdef int i, n
    cdef double dist
    cdef double cost = 0
    n = Y.shape[0]
    for i in range(n):
        dist = d_cython(Y[i,:], C)
        cost += pow(dist,2)
    return cost

def weight_cython(double[:,:]X, double[:,:]C):
    cdef int i, j, n, m, count
    cdef double distx_c, distx_allc
    n = X.shape[0]
    m = C.shape[0]
    w = []
    for j in range(m):
        count = 0
        for i in range(n):
            distx_c = euc_dist_cython(X[i,:],C[j,:])
            distx_allc = d_cython(X[i,:],C)
            if(distx_c == distx_allc):
                count += 1
        w.append(count)
    return w

In [14]:
# K-Means++
def kmeans_plus_cython(X, k):
    idx = np.random.choice(X.shape[0], 1)
    C = X[idx, :]
    while(C.shape[0] < k):
        cost_X = cost_cython(X, C)
        prob = [d_cython(xi,C)**2/cost_X for xi in X]
        Ct = X[np.random.choice(X.shape[0], size=1, p=prob),:]
        if(Ct.tolist() not in C.tolist()):
            C = np.r_[C, Ct]
    return C

In [15]:
###K-means|| speed up
def scalable_kmeans_plus_cython(X, k, l):
    ##Sample a point uniformly at random from X
    idx = np.random.choice(X.shape[0],1)
    C = X[idx,:]
    cost_0 = cost_cython(X, C)
    for i in range(int(round(np.log(cost_0)))):
        cost_X = cost_cython(X,C)
        if(cost_X == 0):
            break
        C = scalable_kmeans_sample(X, C, cost_X, l)
    w = weight_cython(X,C)
    w_prob = w/np.sum(w)
    centroid = np.random.choice(range(C.shape[0]),k,p = w_prob,replace = False)
    return C[centroid]

### Simulating Data

In [16]:
n = 100
mu1 = np.array([2, 6])
cov1 = np.array([[3, 0.5], [0.5, 3]])
subset1 = np.random.multivariate_normal(mu1, cov1, n)
subset1 = np.c_[subset1,np.ones(n)]

mu2 = np.array([5, 2])
cov2 = np.array([[3, 0.5], [0.5, 3]])
subset2 = np.random.multivariate_normal(mu2, cov2, n)
subset2 = np.c_[subset2,np.ones(n)+1]

mu3 = np.array([12, 5])
cov3 = np.array([[3, 1], [1, 3]])
subset3 = np.random.multivariate_normal(mu3, cov3, n)
subset3 = np.c_[subset3,np.ones(n)+2]

data = np.r_[subset1, subset2, subset3]
np.random.shuffle(data)

In [17]:
%%time
kmeans_plus_init = kmeans_plus(data[:,0:2], 3)

CPU times: user 34.9 ms, sys: 1.36 ms, total: 36.2 ms
Wall time: 36.6 ms


In [18]:
%%time
scalable_kmeans_plus_init = scalable_kmeans_plus(data[:,0:2], 3, 2)

CPU times: user 2.37 s, sys: 10.8 ms, total: 2.38 s
Wall time: 2.38 s


In [ ]:
y_pred_kmeans = KMeans(n_clusters=3,init = "random").fit_predict(data[:,0:2])
y_pred_kmeans_plus = KMeans(n_clusters=3,init = kmeans_plus_init, n_init=1).fit_predict(data[:,0:2])
y_pred_scalable_kmeans_plus = KMeans(n_clusters=3,init = scalable_kmeans_plus_init,n_init=1).fit_predict(data[:,0:2])

In [ ]:
plt.scatter(data[:,0], data[:,1], c=data[:,2]);

In [ ]:
plt.scatter(data[:,0], data[:,1], c=y_pred_kmeans);

In [ ]:
plt.scatter(data[:,0], data[:,1], c=y_pred_kmeans_plus);

In [ ]:
plt.scatter(data[:,0], data[:,1], c=y_pred_scalable_kmeans_plus);

### Real Dataset

In [19]:
white_wine = pd.read_csv("winequality-white.csv", sep = ";")
white_wine.quality = white_wine.quality.astype('category')
print(white_wine.shape)
white_wine.head(5)

(4898, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45,170,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14,132,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30,97,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6


In [20]:
white_wine_array = np.array(white_wine, dtype = float)
white_wine_data, white_wine_labels = white_wine_array[:, :-1], white_wine_array[:, -1]

In [21]:
%%time
kmeans_plus_init = kmeans_plus(white_wine_data, 10)

CPU times: user 7.55 s, sys: 65.7 ms, total: 7.61 s
Wall time: 7.78 s


In [22]:
%%time
kmeans_plus_init = kmeans_plus_cython(white_wine_data, 10)

CPU times: user 675 ms, sys: 3.78 ms, total: 678 ms
Wall time: 681 ms


In [23]:
%%time
scalable_kmeans_init = scalable_kmeans_plus(white_wine_data, 10, 2)

CPU times: user 1min 57s, sys: 352 ms, total: 1min 58s
Wall time: 1min 58s


In [24]:
%%time
scalable_kmeans_init = scalable_kmeans_plus_cython(white_wine_data, 10, 2)

CPU times: user 28.1 s, sys: 86.9 ms, total: 28.2 s
Wall time: 28.3 s


In [25]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing as mp

In [35]:
%%time

with mp.Pool(processes=4) as pool:
    res = pool.starmap(scalable_kmeans_plus_cython, [white_wine_data, 10, 2])

TypeError: 'int' object is not iterable

In [ ]:
%%time
scalable_kmeans_init = scalable_kmeans_plus(white_wine_data, 10, 2)

In [ ]:
%%time
scalable_kmeans_init = scalable_kmeans_plus(white_wine_data, 20, 2)